# Full Keras Neural Net

Now that we have a version of our model that integrates Keras into the customer estimator from tf.contrib.learn we can proceed to add in additional code required to train this model on Google Cloud Machine Learning Engine (CMLE).  There are a relatively small number of changes required to get a big uplift in functionality together with the ability to run locally or deploy to CMLE with equal ease from the command line.  Changes will be explained as the solution progresses.

At a high level we do two things:

* Change the mechanism for accessing training/test data from reading directly from local files to reading files from Google Cloud Storage (GCS).
* Encapsulate the model and additional requirements into an Experiment which in turn managed by a learn_runner from tf.contrib.learn.  Experiment has the functionality required to train and evaluate a model in a distributed environment, including checkpointing progress, reporting summaries that can be consumed by TensorBoard, etc.  In turn learn_runner is a utility that helps run and tune an Experiment.

One change from usual practice called out in advance is that usually TensorFlow modelling, especially for CMLE, the model is placed in one file and the associated code to run it is placed in another.  These are usually called model and task.  This is a sensible separation of concerns because it makes it easier to swap models easily.  However, for the purposes of this tutorial I'm combining both into a single notebook/file for convenience.

When you're happy with this proceed to the [next notebook](distributed-keras-local.ipynb).

The next code cell deletes a local directory used when this model is trained locally (i.e. by running this notebook).

In [17]:
!rm -rf abalone_output

In [18]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

import six
from six.moves import urllib

import multiprocessing

import numpy as np

import tensorflow as tf
from tensorflow.python.estimator.model_fn import ModeKeys as Modes

tf.logging.set_verbosity(tf.logging.INFO)

The next section follows standard TensorFlow practices for production grade machine learning.  It defines how to decode a line from a CSV file (together with sensible defaults), instantiates a queue of filenames (in this case we have one), then connects the content of the CSV file to a batching mechanism (that either shuffles the data - usual practice in the case of training - or not - usual practice in the case of evaluation) that encapsulates the details of reading the data, buffering data for training, etc.

In [19]:
CSV_COLUMNS = ['length', 'diameter', 'height', 'whole_weight', 'shucked_weight',
               'viscera_weight', 'shell_weight', 'num_rings']
CSV_COLUMN_DEFAULTS = [[0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0]]

PREDICTED_COLUMN = 'num_rings'
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('length'),
    tf.feature_column.numeric_column('diameter'),
    tf.feature_column.numeric_column('height'),
    tf.feature_column.numeric_column('whole_weight'),
    tf.feature_column.numeric_column('shucked_weight'),
    tf.feature_column.numeric_column('viscera_weight'),
    tf.feature_column.numeric_column('shell_weight'),
]

UNUSED_COLUMNS = set(CSV_COLUMNS) - {col.name for col in INPUT_COLUMNS} - {PREDICTED_COLUMN}

def parse_csv(rows_string_tensor):
  columns = tf.decode_csv(rows_string_tensor, record_defaults=CSV_COLUMN_DEFAULTS)
  features = dict(zip(CSV_COLUMNS, columns))

  for col in UNUSED_COLUMNS:
    features.pop(col)

  for key, value in six.iteritems(features):
    features[key] = tf.expand_dims(features[key], -1)
  return features

def generate_input_fn(filenames,
                      num_epochs=None,
                      shuffle=True,
                      skip_header_lines=0,
                      batch_size=64):
  
  filename_queue = tf.train.string_input_producer(filenames, num_epochs=num_epochs, shuffle=shuffle)
  reader = tf.TextLineReader(skip_header_lines=skip_header_lines)

  _, rows = reader.read_up_to(filename_queue, num_records=batch_size)

  features = parse_csv(rows)

  if shuffle:
    features = tf.train.shuffle_batch(
        features,
        batch_size,
        min_after_dequeue=2 * batch_size + 1,
        capacity=batch_size * 10,
        num_threads=multiprocessing.cpu_count(),
        enqueue_many=True,
        allow_smaller_final_batch=True
    )
  else:
    features = tf.train.batch(
        features,
        batch_size,
        capacity=batch_size * 10,
        num_threads=multiprocessing.cpu_count(),
        enqueue_many=True,
        allow_smaller_final_batch=True
    )

  return features, features.pop(PREDICTED_COLUMN)

In [20]:
from tensorflow.contrib.keras.python.keras.layers import Dense

def generate_model_fn(learning_rate):
    
    def _model_fn(mode, features, labels):

        (length, diameter, height, whole_weight, shucked_weight, viscera_weight, shell_weight) = INPUT_COLUMNS

        transformed_columns = [
            length, diameter, height, whole_weight, shucked_weight, viscera_weight, shell_weight
        ]

        inputs = tf.feature_column.input_layer(features, transformed_columns)

        first_hidden_layer = Dense(10, activation='relu')(inputs)
        second_hidden_layer = Dense(10, activation='relu')(first_hidden_layer)
        output_layer = Dense(1, activation='linear')(second_hidden_layer)

        if mode in (Modes.PREDICT, Modes.EVAL):
          predictions = tf.reshape(output_layer, [-1])
          predictions_dict = {"ages": predictions}

        if mode in (Modes.TRAIN, Modes.EVAL):
          loss = tf.losses.mean_squared_error(labels, output_layer)

        if mode == Modes.TRAIN:
          train_op = tf.contrib.layers.optimize_loss(
            loss=loss,
            global_step=tf.contrib.framework.get_global_step(),
            learning_rate=learning_rate,
            optimizer="SGD")
        
        if mode == Modes.TRAIN:
          return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)
        
        if mode == Modes.EVAL:
          eval_metric_ops = {
              "rmse": tf.metrics.root_mean_squared_error(
                  tf.cast(labels, tf.float32), predictions)
          }
          return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=eval_metric_ops)
        
        if mode == Modes.PREDICT:
          export_outputs = {
              'prediction': tf.estimator.export.RegressionOutput(predictions)
          }
          return tf.estimator.EstimatorSpec(
              mode, predictions=predictions_dict, export_outputs=export_outputs)
    
    return _model_fn

The next section encapsulates producing the Experiment.  Note that this follows standard practice for tf.contrib.learn of providing data via input_fn.

In [21]:
def generate_experiment_fn(**experiment_args):  
  
  def _experiment_fn(run_config, hparams):

    train_input = lambda: generate_input_fn(
      hparams.train_files,
      num_epochs=hparams.num_epochs,
      batch_size=hparams.train_batch_size,
    )

    test_input = lambda: generate_input_fn(
      hparams.eval_files,
      shuffle=False
    )

    return tf.contrib.learn.Experiment(
        tf.estimator.Estimator(
            generate_model_fn(learning_rate=hparams.learning_rate),
            config=run_config
        ),
        train_input_fn=train_input,
        eval_input_fn=test_input,
        **experiment_args
    )
  
  return _experiment_fn

In [22]:
def example_serving_input_fn():
  """Build the serving inputs."""
  example_bytestring = tf.placeholder(
      shape=[None],
      dtype=tf.string,
  )
  features = tf.parse_example(
      example_bytestring,
      tf.feature_column.make_parse_example_spec(INPUT_COLUMNS)
  )
  return tf.estimator.export.ServingInputReceiver(
      features, {'example_proto': example_bytestring})

The next section adds a main method to our code because when we come to run this on CMLE we will change from Datalab to script files.

In [23]:
import argparse
import os

import tensorflow as tf
from tensorflow.contrib.learn import learn_runner
from tensorflow.contrib.learn.python.learn.utils import (
    saved_model_export_utils)
from tensorflow.contrib.training.python.training import hparam

if __name__ == '__main__':
    
    parser = argparse.ArgumentParser()
    # Input Arguments
    parser.add_argument(
        '--train-files',
        help='GCS or local paths to training data',
        nargs='+',
        required=True
    )
    parser.add_argument(
        '--num-epochs',
        help="""\
        Maximum number of training data epochs on which to train.
        If both --max-steps and --num-epochs are specified,
        the training job will run for --max-steps or --num-epochs,
        whichever occurs first. If unspecified will run for --max-steps.\
        """,
        type=int,
    )
    parser.add_argument(
        '--train-batch-size',
        help='Batch size for training steps',
        type=int,
        default=1
    )
    parser.add_argument(
        '--eval-batch-size',
        help='Batch size for evaluation steps',
        type=int,
        default=1
    )
    parser.add_argument(
        '--eval-files',
        help='GCS or local paths to evaluation data',
        nargs='+',
        required=True
    )
    # Training arguments
    parser.add_argument(
        '--learning-rate',
        help='Learning rate for the optimizer',
        default=0.001,
        type=float
    )
    parser.add_argument(
        '--job-dir',
        help='GCS location to write checkpoints and export models',
        required=True
    )
    parser.add_argument(
        '--verbosity',
        choices=[
            'DEBUG',
            'ERROR',
            'FATAL',
            'INFO',
            'WARN'
        ],
        default='INFO',
        help='Set logging verbosity'
    )
    # Experiment arguments
    parser.add_argument(
        '--eval-delay-secs',
        help='How long to wait before running first evaluation',
        default=10,
        type=int
    )
    parser.add_argument(
        '--min-eval-frequency',
        help='Minimum number of training steps between evaluations',
        default=1,
        type=int
    )
    parser.add_argument(
        '--train-steps',
        help="""\
        Steps to run the training job for. If --num-epochs is not specified,
        this must be. Otherwise the training job will run indefinitely.\
        """,
        type=int
    )
    parser.add_argument(
        '--eval-steps',
        help="""\
        Number of steps to run evalution for at each checkpoint.
        If unspecified will run until the input from --eval-files is exhausted
        """,
        default=None,
        type=int
    )

    args = parser.parse_args([
        '--train-files', 'gs://smiling-beaming-abalone/abalone_train.csv',
        '--eval-files', 'gs://smiling-beaming-abalone/abalone_test.csv',
        '--job-dir', 'abalone_output',
        '--train-steps', '5000',
        '--eval-steps', '100'
      ])
    
    #args = parser.parse_args()

    # Set python level verbosity
    tf.logging.set_verbosity(args.verbosity)
    # Set C++ Graph Execution level verbosity
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = str(
        tf.logging.__dict__[args.verbosity] / 10)

    # Run the training job
    # learn_runner pulls configuration information from environment
    # variables using tf.learn.RunConfig and uses this configuration
    # to conditionally execute Experiment, or param server code
    learn_runner.run(
        generate_experiment_fn(
            min_eval_frequency=args.min_eval_frequency,
            eval_delay_secs=args.eval_delay_secs,
            train_steps=args.train_steps,
            eval_steps=args.eval_steps,
            export_strategies=[saved_model_export_utils.make_export_strategy(
                example_serving_input_fn,
                exports_to_keep=1
            )]
        ),
        run_config=tf.contrib.learn.RunConfig(model_dir=args.job_dir),
        hparams=hparam.HParams(**args.__dict__)
    )

INFO:tensorflow:Using config: {'_model_dir': 'abalone_output', '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd72acd5150>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': '', '_session_config': None}
Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into abalone_output/model.ckpt.
INFO:tensorflow:loss = 38.4861, step = 1
INFO:tensorflow:Starting evaluation at 2017-08-02-17:36:58
INFO:tensorflow:Restoring parameters from abalone_output/model.ckpt-1
INFO:tensorflow: